In [1]:
from utils import FakeTRTFunc
from model import load_as_trt_model
import chess.syzygy as syzygy

use_fake = True
if use_fake:
    trt_func = FakeTRTFunc()
else:
    trt_func, _ = load_as_trt_model()
tablebase = syzygy.open_tablebase("/home/tomaz/ChessBot_v2/syzygy/3-4-5")

2025-01-22 21:25:15.326046: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from datetime import datetime
import time
# Create the first timestamp
timestamp1 = datetime.now()
formatted_timestamp1 = timestamp1.strftime("%d/%m %H:%M:%S")
print("First timestamp:", formatted_timestamp1)

# Sleep for 5 seconds
time.sleep(5)

# Create the second timestamp
timestamp2 = datetime.now()
formatted_timestamp2 = timestamp2.strftime("%d/%m %H:%M:%S")
print("Second timestamp:", formatted_timestamp2)

# Calculate the difference
time_difference = timestamp2 - timestamp1
hours, remainder = divmod(time_difference.total_seconds(), 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Difference: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds")

First timestamp: 25/01 19:36:18
Second timestamp: 25/01 19:36:23
Difference: 0 hours, 0 minutes, 5 seconds


In [2]:
from selfplay import play_game
from mcts import MCTS
from configs import selfplayConfig

mctsSearch = MCTS(selfplayConfig)
_, (_, _) = play_game(mctsSearch, trt_func, tablebase, 2)


  +---+---+---+---+---+---+---+---+
  |   | k |   |   |   |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | P |   |   | B | 4
  +---+---+---+---+---+---+---+---+
  |   |   | K |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | R |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 1k6/8/8/8/4P2B/2K5/4R3/8 b - - 0 171
Key: b05c9d7d7042536d
White wins by tablebase in 341 moves in 33.79 seconds


In [1]:
from selfplay import run_selfplay
from mcts import MCTS
from configs import selfplayConfig

mctsSearch = MCTS(selfplayConfig)

""" run_selfplay(
    num_agents=1,
    num_games=3,
    buffer_size=128,
    use_fake_model=True,
    verbose=2,
) """

2025-01-25 19:31:38.696908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


' run_selfplay(\n    num_agents=1,\n    num_games=3,\n    buffer_size=128,\n    use_fake_model=True,\n    verbose=2,\n) '

In [ ]:
from model import reshape_planes
from utils import convert_u64_to_np
from rchess import Board
import numpy as np
import tensorflow as tf

board = Board()
board.push_uci("e2e4")
board.push_uci("e7e5")
board.push_uci("g1f3")
board.push_uci("g8f6")
board.push_uci("e1e2")
#print(board)

history, hash = board.history(False)
np_array = convert_u64_to_np(history)
# Convert the numpy array to a tensorflow tensor
tf_history = tf.convert_to_tensor(np.array([history]), dtype=tf.int64)

# Reshape the tensor using reshape_planes
tf_tensor = reshape_planes(tf_history)
tf_tensor_np = tf_tensor.numpy()
# Check if both are the same
print(np_array.shape, tf_tensor_np.shape)
# Manually loop through the 109 planes and find the differences
differences = []
for i in range(109):
    if not np.array_equal(np_array[0][i], tf_tensor_np[0][i]):
        differences.append(i)

# Print the indices where differences were found
print("Differences found at planes:", differences)
""" plane = 3
print(np_array[0][plane])
print()
print(tf_tensor_np[0][plane]) """
for index in differences:
    print(np_array[0][index])
    print()
    print(tf_tensor_np[0][index])


In [18]:
from mcts import MCTS
from utils import FakeTRTFunc
from rchess import Board
from configs import selfplayConfig
from actionspace import map_w
import numpy as np

trt_func = FakeTRTFunc()
mcts = MCTS(selfplayConfig)

board = Board()
move, root, statistics = mcts.find_best_move(board,
                                             None,
                                             trt_func, 
                                             800,
                                             0.0,
                                             1)

legal_moves = board.legal_moves_uci()
legal_actions = [map_w[move] for move in legal_moves]
print(legal_moves)
print(legal_actions)
assert len(legal_moves) == len(legal_actions) == 20

assert len(statistics) == 1858
for i in range(len(statistics)):
    if i in legal_actions:
        assert statistics[i] > 0 and statistics[i] <= 1
    else:
        assert statistics[i] == 0

assert np.sum(statistics) == 1


['b1a3', 'b1c3', 'g1f3', 'g1h3', 'a2a3', 'b2b3', 'c2c3', 'd2d3', 'e2e3', 'f2f3', 'g2g3', 'h2h3', 'a2a4', 'b2b4', 'c2c4', 'd2d4', 'e2e4', 'f2f4', 'g2g4', 'h2h4']
[222, 221, 1453, 1452, 23, 224, 452, 701, 954, 1207, 1455, 1681, 28, 232, 460, 709, 962, 1215, 1463, 1686]
